In [1]:
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
#import quandl as qd
import seaborn as sns
#from pandas_datareader import data
%matplotlib inline

def save_xls(list_dfs, xls_path, sheet_names):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer, sheet_names[n])
    writer.save()
    return

# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
#tickers = ['VXX', 'SPY']

# Define which online source one should use
#data_source = 'google'

# We would like all available data from 01/01/2000 until 12/31/2016.
#start_date = '2017-01-01'
#end_date = '2017-08-03'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
#panel_data = data.DataReader(tickers, data_source, start_date, end_date)

# panel_data.minor_xs('SPY')

vix = pd.read_csv('VIX.csv')
vix = vix.set_index(pd.DatetimeIndex(vix['Date']))
vix = vix[['Adj Close']]
vix.columns = ['vix']

spx = pd.read_csv('SPX.csv')
spx = spx.set_index(pd.DatetimeIndex(spx['Date']))
spx = spx[['Adj Close']]
spx.columns = ['spx']


In [2]:
# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
# close = panel_data.ix['Close']

# Getting all weekdays between 01/01/2000 and 12/31/2016
#all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex close using all_weekdays as the new index
#close = close.reindex(all_weekdays)

close = pd.concat([vix,spx],axis = 1).dropna()
close.head(10)

,vix,spx
Date,,
1990-01-02,17.240000,359.690002
1990-01-03,18.190001,358.760010
1990-01-04,19.219999,355.670013
1990-01-05,20.110001,352.200012
1990-01-08,20.260000,353.790009
1990-01-09,22.200001,349.619995
1990-01-10,22.440001,347.309998
1990-01-11,20.049999,348.529999
1990-01-12,24.639999,339.929993


In [3]:
def worst_return(vixlvl, dte):
    temp_df = close.copy()
    temp_df['spx_shift'] = temp_df['spx'].shift(-dte)
    temp_df['ret'] = temp_df['spx_shift']/temp_df['spx'] - 1
    temp_df = temp_df[temp_df['vix'] <= vixlvl].dropna()
    print("Worst Return:")
    print(temp_df.loc[temp_df['ret'].idxmin()])
    print("Return Stats:")
    print(temp_df['ret'].describe())
    return temp_df.sort_values('ret', ascending = True)

In [4]:
vxlvl = 14
dte = 60

rets = worst_return(vxlvl,dte)
rets.head(10)

Worst Return:
vix            13.970000
spx          2111.729980
spx_shift    1867.609985
ret            -0.115602
Name: 2015-06-01 00:00:00, dtype: float64
Return Stats:
count    1776.000000
mean        0.023038
std         0.036146
min        -0.115602
25%         0.001457
50%         0.025091
75%         0.045245
max         0.113832
Name: ret, dtype: float64


,vix,spx,spx_shift,ret
Date,,,,
2015-06-01,13.97,2111.729980,1867.609985,-0.115602
2015-05-29,13.84,2107.389893,1893.209961,-0.101633
2015-06-11,12.85,2108.860107,1921.219971,-0.088977
2015-06-18,13.19,2121.239990,1953.030029,-0.079298
2007-05-21,13.30,1525.099976,1406.699951,-0.077634
2007-05-22,13.06,1524.119995,1411.270020,-0.074043
2015-06-10,13.22,2105.199951,1951.130005,-0.073185
2015-06-24,13.26,2108.580078,1958.030029,-0.071399
2015-05-28,13.31,2120.790039,1970.890015,-0.070681
